# Test : Pipe-line vs Dépendances entre variables

Pour en savoir un peu plus sur le pipe-line : https://fr.wikipedia.org/wiki/Pipeline_(architecture_des_processeurs)


Pour voir les effets des conflits entre le pipe-line et les dépendances entre variables, suivons le calcul suivant pour obtenir une multiplication à partir d'additions.

## Multiplication par addition, première version

Pour obtenir une multiplication par 8, 8 additions sont proposées, reposant sur 28 accès mémoire.

In [1]:
%%writefile multiplication.c
#include <stdio.h>
#include <stdlib.h>

int main (int argc, char * argv[]) {
int i,x,y;
if (argc==1)    {i=10;}
else {i=atoi(argv[1]);}
for(x=0,y=1;i;i--) {
  y=y+x;
  y=y+x;
  y=y+x;
  y=y+x;
  y=y+x;
  y=y+x;
  y=y+x;
  y=y+x;
  x=y;}
printf("x=%d, y=%d \n",x,y);
return 0;}

Overwriting multiplication.c


In [2]:
%%sh
arm-linux-gnueabi-gcc -S -static multiplication.c
cat multiplication.s

	.arch armv5t
	.fpu softvfp
	.eabi_attribute 20, 1
	.eabi_attribute 21, 1
	.eabi_attribute 23, 3
	.eabi_attribute 24, 1
	.eabi_attribute 25, 1
	.eabi_attribute 26, 2
	.eabi_attribute 30, 6
	.eabi_attribute 34, 0
	.eabi_attribute 18, 4
	.file	"multiplication.c"
	.section	.rodata
	.align	2
.LC0:
	.ascii	"x=%d, y=%d \012\000"
	.text
	.align	2
	.global	main
	.type	main, %function
main:
	@ args = 0, pretend = 0, frame = 24
	@ frame_needed = 1, uses_anonymous_args = 0
	stmfd	sp!, {fp, lr}
	add	fp, sp, #4
	sub	sp, sp, #24
	str	r0, [fp, #-24]
	str	r1, [fp, #-28]
	ldr	r3, [fp, #-24]
	cmp	r3, #1
	bne	.L2
	mov	r3, #10
	str	r3, [fp, #-16]
	b	.L3
.L2:
	ldr	r3, [fp, #-28]
	add	r3, r3, #4
	ldr	r3, [r3, #0]
	mov	r0, r3
	bl	atoi
	str	r0, [fp, #-16]
.L3:
	mov	r3, #0
	str	r3, [fp, #-12]
	mov	r3, #1
	str	r3, [fp, #-8]
	b	.L4
.L5:
	ldr	r2, [fp, #-8]
	ldr	r3, [fp, #-12]
	add	r3, r2, r3
	str	r3, [fp, #-8]
	ldr	r2, [fp, #-8]
	ldr	r3, [fp, #-12]
	add	r3, r2, r3
	str	r3, [fp, #-8]
	ldr	r2, [fp, #-8]
	ldr	r3, [f

In [28]:
%%sh
arm-linux-gnueabi-gcc -static multiplication.c
qemu-arm a.out

x=387420489, y=387420489 


In [4]:
%%sh
arm-linux-gnueabi-gcc -static multiplication.c
time qemu-arm a.out 60000000

x=-1027070407, y=-1027070407 


0.93user 0.00system 0:00.94elapsed 99%CPU (0avgtext+0avgdata 4128maxresident)k
0inputs+0outputs (0major+1115minor)pagefaults 0swaps


N.B. : attention, les résultats sont à comprendre comme des résultats sur 32 bits (seulement)

## Multiplication par addition, version améliorée par le programmeur

On veut éviter les dépendances entre variables, mais garder le même nombre d'opérations et aussi le même nombre d'accès mémoire. Des variables suppléméntaires sont ajoutées, les calculs utils sont un peu améliorés, des calculs inutils sont ajoutés pour compenser.

In [5]:
%%writefile multiplicationEnArbre.c
#include <stdio.h>
#include <stdlib.h>

int main (int argc, char * argv[]) {
int i,x,y,xx,xxxx,xxxxxxxx,t,tt,ttt,tttt;
if (argc==1)    {i=10;}
else {i=atoi(argv[1]);}
for(x=0,y=1;i;i--) {
    xx=x+x;
    t=x+x;
    xxxx=xx+xx;
    tt=x+x;
    xxxxxxxx=xxxx+xxxx;
    ttt=x+x;
    y=y+xxxxxxxx;    
    tttt=x+x;
    x=y;}
  printf("x=%d, y=%d \n",x,y);
  return 0;}


Overwriting multiplicationEnArbre.c


In [17]:
%%sh
arm-linux-gnueabi-gcc -S -static multiplicationEnArbre.c
cat multiplicationEnArbre.s

	.arch armv5t
	.fpu softvfp
	.eabi_attribute 20, 1
	.eabi_attribute 21, 1
	.eabi_attribute 23, 3
	.eabi_attribute 24, 1
	.eabi_attribute 25, 1
	.eabi_attribute 26, 2
	.eabi_attribute 30, 6
	.eabi_attribute 34, 0
	.eabi_attribute 18, 4
	.file	"multiplicationEnArbre.c"
	.section	.rodata
	.align	2
.LC0:
	.ascii	"x=%d, y=%d \012\000"
	.text
	.align	2
	.global	main
	.type	main, %function
main:
	@ args = 0, pretend = 0, frame = 48
	@ frame_needed = 1, uses_anonymous_args = 0
	stmfd	sp!, {fp, lr}
	add	fp, sp, #4
	sub	sp, sp, #48
	str	r0, [fp, #-48]
	str	r1, [fp, #-52]
	ldr	r3, [fp, #-48]
	cmp	r3, #1
	bne	.L2
	mov	r3, #10
	str	r3, [fp, #-44]
	b	.L3
.L2:
	ldr	r3, [fp, #-52]
	add	r3, r3, #4
	ldr	r3, [r3, #0]
	mov	r0, r3
	bl	atoi
	str	r0, [fp, #-44]
.L3:
	mov	r3, #0
	str	r3, [fp, #-40]
	mov	r3, #1
	str	r3, [fp, #-36]
	b	.L4
.L5:
	ldr	r2, [fp, #-40]
	ldr	r3, [fp, #-40]
	add	r3, r2, r3
	str	r3, [fp, #-32]
	ldr	r2, [fp, #-40]
	ldr	r3, [fp, #-40]
	add	r3, r2, r3
	str	r3, [fp, #-28]
	ldr	r2, [fp, #-32

In [8]:
%%sh
arm-linux-gnueabi-gcc -static multiplicationEnArbre.c
time qemu-arm a.out 60000000

x=-1027070407, y=-1027070407 


0.64user 0.01system 0:00.66elapsed 98%CPU (0avgtext+0avgdata 4124maxresident)k
0inputs+0outputs (0major+1116minor)pagefaults 0swaps


## Arm vs X86 ?

Mais est-ce que qemu simule le pipe-line (ou seulement bénéficie d'un pipe-line de la machine ? ou produit un code qui peut en bénéficiér ?) ? Bref, avec qemu, ce n'est pas tout à fait sûr de savoir ce qui se passe. Pour avoir confirmation de ce que l'on a l'impression d'observer regardons en X86 (cette machine est en X86, pas en ARM) pour un langage compilé (il ne faudrait peut-être pas prendre Python ou java pour voir cela)

In [9]:
%%sh
gcc multiplication.c
time ./a.out 60000000

x=-1027070407, y=-1027070407 


0.81user 0.01system 0:00.81elapsed 101%CPU (0avgtext+0avgdata 504maxresident)k
0inputs+0outputs (0major+157minor)pagefaults 0swaps


In [11]:
%%sh
gcc  multiplicationEnArbre.c
time ./a.out 60000000

x=-1027070407, y=-1027070407 


0.50user 0.00system 0:00.50elapsed 99%CPU (0avgtext+0avgdata 504maxresident)k
0inputs+0outputs (0major+158minor)pagefaults 0swaps


## Conclusion 

On peut ajouter des opérations inutiles et aller plus vite ! (à condition d'améliorer les opérations utiles)  
(on peut même enlever ensuite les opérations inutiles, cela risque d'aller encore plus vite, mais c'est normal, cette fois, il y aura moins de calculs et d'accès mémoire à faire)

In [12]:
%%writefile multiplicationEnArbreDirecte.c
#include <stdio.h>
#include <stdlib.h>

int main (int argc, char * argv[]) {
int i,x,y,xx,xxxx,xxxxxxxx;
if (argc==1)    {i=10;}
else {i=atoi(argv[1]);}
for(x=0,y=1;i;i--) {
    xx=x+x;
    xxxx=xx+xx;
    xxxxxxxx=xxxx+xxxx;
    y=y+xxxxxxxx;    
    x=y;}
  printf("x=%d, y=%d \n",x,y);
  return 0;}


Overwriting multiplicationEnArbreDirecte.c


In [15]:
%%sh
arm-linux-gnueabi-gcc -static multiplicationEnArbreDirecte.c
time qemu-arm a.out 60000000

x=-1027070407, y=-1027070407 


0.51user 0.01system 0:00.53elapsed 99%CPU (0avgtext+0avgdata 4128maxresident)k
0inputs+0outputs (0major+1116minor)pagefaults 0swaps


In [17]:
%%sh
gcc  multiplicationEnArbreDirecte.c
time ./a.out 60000000

x=-1027070407, y=-1027070407 


0.54user 0.00system 0:00.54elapsed 100%CPU (0avgtext+0avgdata 504maxresident)k
0inputs+0outputs (0major+155minor)pagefaults 0swaps
